In [2]:
# import cv2
# import os

# # Define the paths for train and test folders
# train_folder = 'TEST'  # Update with the path to your train folder
# test_folder = 'TRAIN'    # Update with the path to your test folder

# # Haar Cascade for face detection (pre-trained XML file provided by OpenCV)
# haar_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
# face_cascade = cv2.CascadeClassifier(haar_cascade_path)

# # Function for face detection
# def detect_faces(video_path, save_faces=False, output_folder="output_faces"):
#     cap = cv2.VideoCapture(video_path)
#     frame_count = 0

#     # Create the output folder if saving faces
#     if save_faces and not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Convert to grayscale for face detection
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#         # Detect faces
#         faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

#         # Draw rectangles around detected faces
#         for (x, y, w, h) in faces:
#             cv2.rectangle(frame, (x, y), (xq + w, y + h), (255, 0, 0), 2)

#             # Optionally save detected faces as individual images
#             if save_faces:
#                 face = frame[y:y+h, x:x+w]
#                 output_path = os.path.join(output_folder, f"face_{frame_count}.jpg")
#                 cv2.imwrite(output_path, face)

#         # Display the frame with detected faces (optional)
#         cv2.imshow('Face Detection', frame)

#         # Break loop on 'q' key press
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#         frame_count += 1

#     cap.release()
#     cv2.destroyAllWindows()

# # Process all videos in the train folder
# for actor_folder in os.listdir(train_folder):
#     actor_path = os.path.join(train_folder, actor_folder)
#     if os.path.isdir(actor_path):
#         for video_file in os.listdir(actor_path):
#             video_path = os.path.join(actor_path, video_file)
#             print(f"Processing video: {video_path}")
#             detect_faces(video_path, save_faces=True, output_folder="train_faces")

# # Repeat for test folder (optional)
# for actor_folder in os.listdir(test_folder):
#     actor_path = os.path.join(test_folder, actor_folder)
#     if os.path.isdir(actor_path):
#         for video_file in os.listdir(actor_path):
#             video_path = os.path.join(actor_path, video_file)
#             print(f"Processing video: {video_path}")
#             detect_faces(video_path, save_faces=True, output_folder="test_faces")


Processing video: TEST\Actor_21\01-01-01-01-01-01-21.mp4
Processing video: TEST\Actor_21\01-01-01-01-01-02-21.mp4
Processing video: TEST\Actor_21\01-01-01-01-02-01-21.mp4
Processing video: TEST\Actor_21\01-01-01-01-02-02-21.mp4
Processing video: TEST\Actor_21\01-01-02-01-01-01-21.mp4
Processing video: TEST\Actor_21\01-01-02-01-01-02-21.mp4
Processing video: TEST\Actor_21\01-01-02-01-02-01-21.mp4
Processing video: TEST\Actor_21\01-01-02-01-02-02-21.mp4
Processing video: TEST\Actor_21\01-01-02-02-01-01-21.mp4
Processing video: TEST\Actor_21\01-01-02-02-01-02-21.mp4
Processing video: TEST\Actor_21\01-01-02-02-02-01-21.mp4
Processing video: TEST\Actor_21\01-01-02-02-02-02-21.mp4
Processing video: TEST\Actor_21\01-01-03-01-01-01-21.mp4
Processing video: TEST\Actor_21\01-01-03-01-01-02-21.mp4
Processing video: TEST\Actor_21\01-01-03-01-02-01-21.mp4
Processing video: TEST\Actor_21\01-01-03-01-02-02-21.mp4
Processing video: TEST\Actor_21\01-01-03-02-01-01-21.mp4
Processing video: TEST\Actor_21

KeyboardInterrupt: 

In [19]:
import cv2
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix, classification_report
from skimage.feature import local_binary_pattern
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_approximation import AdditiveChi2Sampler

In [20]:
train_folder='TRAIN'
test_folder='TEST'

emotion_mapping={"01": "Neutral","02": "Calm","03": "Happy","04": "Sad","05": "Angry","06": "Fearful","07": "Disgust","08": "Surprised"}



In [21]:
def face_detection(one_frame):
    haar_cascade_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    face_cascade = cv2.CascadeClassifier(haar_cascade_path)
    gray = cv2.cvtColor(one_frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces


In [22]:
def video_processing(video_path):
    video_capture=cv2.VideoCapture(video_path)
    LBP_features=[]
    while video_capture.isOpened():
        ret, frame=video_capture.read()
        if not ret:
            break
        faces=face_detection(frame)  #detecting faces-> faces are the detected faces which can now be used to extract lbp features
        # Extract features for each face
        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (128, 128))
           
            gray = cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY)
            lbp = local_binary_pattern(gray, P=8, R=1, method="uniform")
            hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 27), density=True) #hist is the lbp feature extracted
            LBP_features.append(hist)
    video_capture.release()
    return np.mean(LBP_features, axis=0) if LBP_features else np.zeros((26,))



        

In [25]:
x=[]
y=[]
max_vids=3
current=0
for actor_folder in os.listdir(train_folder):
    if current >= max_vids:  # Stop once the total videos processed reaches the limit
        break
    actor_path=os.path.join(train_folder, actor_folder)
    if  os.path.isdir(actor_path):
        for video in os.listdir(actor_path):
                if current >= max_vids:  # Stop once the total videos processed reaches the limit
                  break
                video_path_train=os.path.join(actor_path, video)
                emotion_number=video.split("-")[2]
                emotion_value=emotion_mapping[emotion_number]
                print(f"Processing video: {video_path_train}")
                features_after_processing=video_processing(video_path_train)
                x.append(features_after_processing)
                y.append(emotion_value)
                current+=1

Processing video: TRAIN\Actor_01\01-01-01-01-01-01-01.mp4
Processing video: TRAIN\Actor_01\01-01-01-01-01-02-01.mp4
Processing video: TRAIN\Actor_01\01-01-01-01-02-01-01.mp4


In [26]:
X_train=np.array(x)
Y_train=np.array(y)

In [28]:
print(X_train)

[[0.03758894 0.05170425 0.05090768 0.11993097 0.22922002 0.16487215
  0.06125376 0.06786673 0.12077985 0.09587564 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.03753724 0.05260861 0.05095757 0.12218314 0.23291324 0.16256621
  0.06137609 0.06726999 0.11663263 0.09595528 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.04227786 0.05347183 0.05225176 0.11857495 0.22148714 0.15978438
  0.06236535 0.06952282 0.11800361 0.10226031 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]]
